In [41]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score,precision_score, recall_score, roc_auc_score
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.mixture import GaussianMixture
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
import tensorflow.keras
import keras.metrics
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Input
from keras.models import Sequential
from keras.callbacks import History 
from keras.utils import plot_model
from keras.optimizers import SGD
from keras.models import Model
from imblearn.under_sampling import RandomUnderSampler
from sklearn.semi_supervised import LabelSpreading, LabelPropagation


In [42]:
path = "PTSD.xlsx"
df = pd.read_excel(path)
#df = df[~df["PCL_Strict3"].isna()]


In [43]:
features = [ "age", "highschool_diploma", "Hebrew", "dyslexia", "ADHD", "T1ETBE", "T1Acc1t",
                         "T1Acc1n", "T1bias", "T2Acc1t", "T2Acc1n", "T2bias", "state1", "state2", "trait1",
                         "trait2", "lot1", "lot2", "phq1", "phq2", "PCL1", "PCL2", "cd_risc1", "ptgi2",
                         "active_coping1", "planning1", "positive_reframing1", "acceptance1", "humor1",
                         "religion1", "emotional_support1", "instrumental_support1", "self_distraction1",
                         "denial1", "venting1", "substance_use1", "behavioral_disengagement1", "self_blame1",
                         "active_coping2", "planning2", "positive_reframing2", "acceptance2", "humor2",
                         "religion2", "emotional_support2", "instrumental_support2","self_distraction2",
                         "denial2", "venting2", "substance_use2", "behavioral_disengagement2", "self_blame2",
                         "trauma_history8_1", "military_exposure_unit", "HML_5HTT", "HL_MAOA", "HML_NPY",
                         "COMT_Hap1_recode", "COMT_Hap2_recode", "COMT_Hap1_LvsMH", "HML_FKBP5", "Ashken_scale",
                         "Sephar_scale", "Unknown",
                        'terror_p1', 'terror_i1', 'mva_p1', 'mva_i1',                   
                        'violent1', 'sexual1', 'rockets_p1', 'rockets_i1', 'trauma_history6_1',
                        'terror_p2','terror_i2','mva_p2', 'mva_i2', 'violent2', 'sexual2', 'rockets_p2', 
                        'rockets_i2', 'trauma6t2', 'trauma8t2', 'military_exp18_1','military_exp18_t2',
                        'commanders18','commanders20','commanders22', 
                        't1bias_1_zero', 'state1_zero', 'trait1_zero', 'PHQ1_zero', 'PCL1_zero',
                        'depression_clinical2', 'avoid_bias', 'ptsd1_clini', 'avoidance_cop',
                        'clinical_depression15', 'avoidance_compa', 'resilience_compa', 'combat_compa',
                        'emotional_cop1n', 'avoidance_cop2', 'avoidance_cop']

In [44]:
numerical_features = [ "T1ETBE", "T1Acc1t", "T1Acc1n", "T1bias", "T2Acc1t","T2Acc1n", "T2bias", "state1", "state2",
                        "trait1", "trait2", "lot1", "lot2", "phq1", "phq2", "cd_risc1", "PCL1", "PCL2"]
categorical_features = [ "age", "highschool_diploma", "Hebrew", "dyslexia", "ADHD",   "ptgi2",
                    "active_coping1", "planning1", "positive_reframing1", "acceptance1", "humor1",
                    "religion1", "emotional_support1", "instrumental_support1", "self_distraction1",
                    "denial1", "venting1", "substance_use1", "behavioral_disengagement1", "self_blame1",
                    "active_coping2", "planning2", "positive_reframing2", "acceptance2", "humor2",
                    "religion2", "emotional_support2", "instrumental_support2", "self_distraction2",
                    "denial2", "venting2", "substance_use2", "behavioral_disengagement2", "self_blame2",
                    "trauma_history8_1", "military_exposure_unit", "HML_5HTT", "HL_MAOA", "HML_NPY",
                    "COMT_Hap1_recode", "COMT_Hap2_recode", "COMT_Hap1_LvsMH", "HML_FKBP5", "Ashken_scale",
                    "Sephar_scale", "Unknown", 'terror_p1', 'terror_i1', 'mva_p1', 'mva_i1',                   
                        'violent1', 'sexual1', 'rockets_p1', 'rockets_i1', 'trauma_history6_1',
                        'terror_p2','terror_i2','mva_p2', 'mva_i2', 'violent2', 'sexual2', 'rockets_p2', 'rockets_i2', 'trauma6t2', 'trauma8t2',
'military_exp18_1','military_exp18_t2', 'commanders18','commanders20','commanders22', 
't1bias_1_zero', 'state1_zero', 'trait1_zero', 'PHQ1_zero', 'PCL1_zero', 'depression_clinical2',
                        'avoid_bias', 'd_ptsd', 'ptsd1_clini', 'avoidance_cop',
'clinical_depression15', 'avoidance_compa', 'resilience_compa', 'combat_compa', 'emotional_cop1n', 'avoidance_cop2', 'avoidance_cop']
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
df[numerical_features] = imp.fit_transform(df[numerical_features])

imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df[categorical_features] = imp.fit_transform(df[categorical_features])


df[features] = df[features] - df[features].mean()

Y = df[~df["PCL_Strict3"].isna()]["PCL_Strict3"]

In [45]:
df["interaction_1"] = df["T1Acc1t"] * df["T2Acc1n"] * df["military_exposure_unit"]
df["interaction_2"] = df["T1Acc1n"] * df["T2Acc1t"] * df["military_exposure_unit"]
df["interaction_3"] = df["highschool_diploma"] * df["military_exposure_unit"] * df['PCL1']
df["interaction_4"] = df["T1ETBE"] * df["military_exposure_unit"] * df['HML_5HTT']
deltas = ["state", "trait", "lot", "phq",  "PCL",
                         "active_coping", "planning", "positive_reframing", "acceptance", "humor",
                         "religion", "emotional_support", "instrumental_support", "self_distraction",
                         "denial", "venting", "substance_use", "behavioral_disengagement", "self_blame"]
abm = ["T1Acc1t", "T1Acc1n", "T1bias"]
for feature in abm:
    df[feature[2::]+"_delta"] = (df["T1"+feature[2::]] - df["T2"+feature[2::]])
for feature in deltas:
    df[feature+"_delta"] = (df[feature + "1"] - df[feature + "2"])

In [46]:
X = df[~df["PCL_Strict3"].isna()][features]

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.1, random_state=271828, stratify=Y)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_train, y_train, test_size = 0.05, random_state=271828, stratify=y_train)


In [48]:
def create_data(X_train, y_train):
    X_train_3 = X_train[y_train==1]
    y_train_3 = y_train[y_train==1]
    X_train_4 = X_train[y_train==0][:20:]
    y_train_4 = y_train[y_train==0][:20:]
    X_train_5 = np.vstack((X_train_4, X_train_3))
    y_train_5 =  np.hstack((y_train_4, y_train_3))
    sm = SMOTE(random_state=27)
    X_train_6, y_train_6 = sm.fit_sample(X_train_5, y_train_5.ravel())
    X_train_6 = X_train_6[y_train_6==0]
    y_train_6 = y_train_6[y_train_6==0]
    return X_train_6, y_train_6

In [49]:
def semi(X_train, y_train, X_test, pca):
    
    xxx = df[df["PCL_Strict3"].isna()][features]
    xxx = pca.transform(xxx)
    X_train_2 = np.vstack((X_train, xxx))
    
    unlabeled = df[df["PCL_Strict3"].isna()]['PCL_Strict3'].fillna(-1)
    
    y_train_2 = np.hstack((y_train, unlabeled))
    
    clf = LabelSpreading(gamma=20, n_neighbors=150,alpha=0.0005,kernel='knn')
    
    clf.fit(X_train_2, y_train_2)
    
    
    X_train = np.hstack((X_train, clf.predict(X_train).reshape(-1,1)))
    X_test = np.hstack((X_test, clf.predict(X_test).reshape(-1,1)))
    return X_train, X_test

In [50]:
def auto(X_train, X_test, pca):
    
    xxx = df[df["PCL_Strict3"].isna()][features]
    xxx = pca.transform(xxx)
    X_train_2 = np.vstack((X_train, xxx))

    encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

    # this is our input placeholder
    input_img = Input(shape=(X_train_2.shape[1],))
    # "encoded" is the encoded representation of the input
    encoded = Dense(encoding_dim, activation='relu')(input_img)
    # "decoded" is the lossy reconstruction of the input
    decoded = Dense(X_train_2.shape[1], activation='sigmoid')(encoded)

    # this model maps an input to its reconstruction
    autoencoder = Model(input_img, decoded)

    encoder = Model(input_img, encoded)
    # create a placeholder for an encoded (32-dimensional) input
    encoded_input = Input(shape=(encoding_dim,))
    # retrieve the last layer of the autoencoder model
    decoder_layer = autoencoder.layers[-1]
    # create the decoder model
    decoder = Model(encoded_input, decoder_layer(encoded_input))
    autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
    autoencoder.fit(X_train, X_train,
                    epochs=500,
                    batch_size=256,
                    shuffle=True, verbose=0)    

    X_train = np.hstack((X_train, encoder.predict(X_train)))
    X_test = np.hstack((X_test, encoder.predict(X_test)))
    return X_train, X_test

In [51]:

from keras.datasets import mnist 
from keras.models import Model 
from keras.layers import Input, Dense 
from keras.utils import np_utils 
import numpy as np


In [53]:
kfold = StratifiedKFold(n_splits=3, shuffle=True)
cvscores = []
y_train_2 = np.array(y_train_2)
X_train_2 = np.array(X_train_2)

for class_weight in [{1:0.6, 0:0.4}, {1:0.57, 0:0.43}]:
    for num_smote in [0.95, 0.99]:
        for lim in [0.5]:
            for first_layer in [75]:
                for loops in [1]:
                    for each_layer in [11]:
                        num_layers = 2
                        n_pca = 0.99
                        first_layer = first_layer
                        each_layer = each_layer
                        num_smote = num_smote
                        loops = loops
                        scores_f = []
                        scores_p = []
                        scores_r = []
                        print("\n\nlim", lim, "\nfirst_layer", first_layer, 
                                  "\neach_layer", each_layer, "\nnum_smote", num_smote,
                              "\nloops", loops, "\nclass_weight", class_weight)
                        for train, test in kfold.split(X_train_2, y_train_2):
                            
                            y_pred = np.zeros_like(y_train_2[test]).reshape(-1, 1)
                            t = 5
                            for depth in range(t):
                                sm = RandomUnderSampler(num_smote)
                                X_train_res, y_train_res = sm.fit_sample(X_train_2[train], y_train_2[train].ravel())


                                n_cols = X_train_res.shape[1]

                                input_img = Input(shape=(n_cols,))

                                x = Dense(n_cols, activation='relu')(input_img)

                                encoded = Dense(n_cols//2, activation='relu')(x)
                                encoded = Dense(n_cols//4, activation='relu')(encoded)

                                y = Dense(n_cols//8, activation='relu')(x)

                                decoded = Dense(n_cols//4, activation='relu')(y)
                                decoded = Dense(n_cols//2, activation='relu')(decoded)

                                z = Dense(n_cols, activation='sigmoid')(decoded)
                                model = Model(input_img, z)

                                model.compile(optimizer='adadelta', loss='mse') # reporting the accuracy

                                xxx = df[df["PCL_Strict3"].isna()][features]
                                X_train_res_2 = np.vstack((X_train_res, xxx))


                                model.fit(X_train_res_2, X_train_res_2,
                                      epochs=1000,
                                      batch_size=128,
                                      verbose=0)

                                out = Dense(1, activation='sigmoid')(y)
                                reduced = Model(input_img, out)
                                reduced.compile(loss='binary_crossentropy',
                                          optimizer='adam', 
                                          metrics=['accuracy']) 

                                reduced.fit(X_train_res, y_train_res,
                                      epochs=1000,
                                      batch_size=128,
                                      verbose=0, class_weight=class_weight)

                                y_pred +=  reduced.predict(X_train_2[test])

                        
                            y_pred = (y_pred/t) > lim
                            s_f = f1_score(y_pred,y_train_2[test])
                            s_p = precision_score(y_pred,y_train_2[test])
                            s_r = recall_score(y_pred,y_train_2[test])
                            print("\tscores f1", (s_f))
                            print("\tscores p", (s_p))
                            print("\tscores r", (s_r))
                            scores_f.append(s_f)
                            scores_p.append(s_p)
                            scores_r.append(s_r)

                        print("mean scores f1", np.mean(scores_f))
                        print("mean scores p", np.mean(scores_p))
                        print("mean scores r", np.mean(scores_r))



lim 0.5 
first_layer 75 
each_layer 11 
num_smote 0.95 
loops 1 
class_weight {1: 0.6, 0: 0.4}
	scores f1 0.17391304347826086
	scores p 0.6153846153846154
	scores r 0.10126582278481013


KeyboardInterrupt: 

In [15]:
kfold = StratifiedKFold(n_splits=3, shuffle=True)
cvscores = []
y_train_2 = np.array(y_train_2)
X_train_2 = np.array(X_train_2)

for class_weight in [{1:0.61, 0:0.39}, {1:0.55, 0:0.45}]:
    for num_smote in [0.95, 0.9]:
        for num_layers in [3]:
            for first_layer in [75]:
                for loops in [50]:
                    for each_layer in [12]:
                        num_layers = num_layers
                        first_layer = first_layer
                        each_layer = each_layer
                        num_smote = num_smote
                        loops = loops
                        scores_f = []
                        scores_p = []
                        scores_r = []
                        print("\n\nnum_layers", num_layers, "\nfirst_layer", first_layer, 
                                  "\neach_layer", each_layer, "\nnum_smote", num_smote,
                              "\nloops", loops, "\nclass_weight", class_weight)
                        for train, test in kfold.split(X_train_2, y_train_2):

                            x, y = create_data(X_train_2[train], y_train_2[train])
                            created_X = x
                            created_y = y
                            for i in range(loops):
                                X_train_3, y_train_3 = shuffle(X_train_2[train], y_train_2[train])
                                x, y = create_data(X_train_3, y_train_3)
                                created_X = np.vstack((created_X, x))
                                created_y = np.hstack((created_y, y))
                            X_train_3 = np.vstack([created_X, X_train_3])
                            y_train_3 =  np.hstack([created_y, y_train_3])

                            sm = SMOTE(random_state=27)
                            X_train_res, y_train_res = sm.fit_sample(X_train_3, y_train_3.ravel())


                            n_cols = X_train_res.shape[1]

                            input_img = Input(shape=(n_cols,))

                            x = Dense(n_cols, activation='relu')(input_img)

                            encoded = Dense(n_cols//2, activation='relu')(x)
                            encoded = Dense(n_cols//4, activation='relu')(encoded)

                            y = Dense(n_cols//8, activation='relu')(x)

                            decoded = Dense(n_cols//4, activation='relu')(y)
                            decoded = Dense(n_cols//2, activation='relu')(decoded)

                            z = Dense(n_cols, activation='sigmoid')(decoded)
                            model = Model(input_img, z)

                            model.compile(optimizer='adadelta', loss='mse') # reporting the accuracy

                            xxx = df[df["PCL_Strict3"].isna()][features]
                            X_train_res_2 = np.vstack((X_train_res, xxx))

                            
                            model.fit(X_train_res_2, X_train_res_2,
                                  epochs=1000,
                                  batch_size=128,
                                  verbose=0)

                            out = Dense(1, activation='sigmoid')(y)
                            reduced = Model(input_img, out)
                            reduced.compile(loss='binary_crossentropy',
                                      optimizer='adam', 
                                      metrics=['accuracy']) 

                            reduced.fit(X_train_res, y_train_res,
                                  epochs=1000,
                                  batch_size=128,
                                  verbose=0, class_weight=class_weight)

                            y_pred =  reduced.predict(X_train_2[test])
                        
                            
                            y_pred = (y_pred) > 0.5
                            
                            s_f = f1_score(y_pred,y_train_2[test])
                            s_p = precision_score(y_pred,y_train_2[test])
                            s_r = recall_score(y_pred,y_train_2[test])
                            print("\tscores f1", (s_f))
                            print("\tscores p", (s_p))
                            print("\tscores r", (s_r))
                            scores_f.append(s_f)
                            scores_p.append(s_p)
                            scores_r.append(s_r)

                        print("mean scores f1", np.mean(scores_f))
                        print("mean scores p", np.mean(scores_p))
                        print("mean scores r", np.mean(scores_r))
                       



num_layers 3 
first_layer 75 
each_layer 12 
num_smote 0.95 
loops 50 
class_weight {1: 0.61, 0: 0.39}
	scores f1 0.08
	scores p 0.07692307692307693
	scores r 0.08333333333333333
	scores f1 0.3
	scores p 0.23076923076923078
	scores r 0.42857142857142855
	scores f1 0.08333333333333333
	scores p 0.08333333333333333
	scores r 0.08333333333333333
mean scores f1 0.15444444444444444
mean scores p 0.13034188034188035
mean scores r 0.1984126984126984


num_layers 3 
first_layer 75 
each_layer 12 
num_smote 0.9 
loops 50 
class_weight {1: 0.61, 0: 0.39}


C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


	scores f1 0.0
	scores p 0.0
	scores r 0.0
	scores f1 0.3076923076923077
	scores p 0.3076923076923077
	scores r 0.3076923076923077
	scores f1 0.09523809523809525
	scores p 0.08333333333333333
	scores r 0.1111111111111111
mean scores f1 0.1343101343101343
mean scores p 0.13034188034188035
mean scores r 0.1396011396011396


num_layers 3 
first_layer 75 
each_layer 12 
num_smote 0.95 
loops 50 
class_weight {1: 0.55, 0: 0.45}
	scores f1 0.24000000000000002
	scores p 0.23076923076923078
	scores r 0.25


KeyboardInterrupt: 